In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

path = r"C:\Users\Gabriel\Documents\Inge-2I\UTK_Dataset"
pixels = []
age = []
gender = []
for img in os.listdir(path):
  ages = img.split("_")[0]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  pixels.append(np.array(img))
  age.append(np.array(ages))
age = np.array(age,dtype=np.int64)
pixels = np.array(pixels)

x_train,x_test,y_train,y_test = train_test_split(pixels,age,random_state=100)

input = Input(shape=(200,200,3))
conv1 = Conv2D(140,(3,3),activation="relu")(input)
conv2 = Conv2D(130,(3,3),activation="relu")(conv1)
batch1 = BatchNormalization()(conv2)
pool3 = MaxPool2D((2,2))(batch1)
conv3 = Conv2D(120,(3,3),activation="relu")(pool3)
batch2 = BatchNormalization()(conv3)
pool4 = MaxPool2D((2,2))(batch2)
flt = Flatten()(pool4)
#age
age_l = Dense(128,activation="relu")(flt)
age_l = Dense(64,activation="relu")(age_l)
age_l = Dense(32,activation="relu")(age_l)
age_l = Dense(1,activation="relu")(age_l)

checkpoint_callback = ModelCheckpoint(
    filepath='AgePersonelModel2.keras',
    monitor='val_mae',
    save_best_only=True,
    save_weights_only=False,
    mode='min',  # 'min' car nous voulons minimiser 'val_mae'
    verbose=1
)


model = Model(inputs=input,outputs=[age_l])
model.compile(optimizer="adam",loss=["mse"],metrics=['mae'])
save = model.fit(x_train,[y_train],validation_data=(x_test,[y_test]),epochs=50,callbacks=[checkpoint_callback])
model.save("model.keras")